<a href="https://colab.research.google.com/github/csabiu/Astrostatistics/blob/main/GAIA_cepheids.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Install some packages and import them

In [ ]:
pip install astroquery

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
from astropy.coordinates import (SkyCoord, Distance, Galactic,
                                 EarthLocation, AltAz)
import astropy.coordinates as coord
from astropy.time import Time
from astropy import units as u
from astropy.coordinates import AltAz
from astropy.io import fits
from astropy.table import QTable
from astropy.utils.data import download_file
from astroquery.gaia import Gaia


launch SQL job on GAIA archive

see the below website for all available columns and descriptions:

https://gea.esac.esa.int/archive/

In [ ]:
job = Gaia.launch_job_async("SELECT gaia.source_id,gaia.ref_epoch,gaia.ra,gaia.dec,gaia.l,gaia.b,gaia.parallax,gaia.parallax_error,gaia.pmra,gaia.pmra_error,gaia.pmdec,gaia.pmdec_error, gaia.phot_g_n_obs,gaia.phot_g_mean_mag,gaia.bp_rp,gaia.radial_velocity,gaia.radial_velocity_error, gaia.rv_template_teff, cep.pf,cep.pf_error, gaia.ag_gspphot , cep.mode_best_classification "
                        "FROM gaiadr3.gaia_source AS gaia "
                        "JOIN gaiadr3.vari_cepheid AS cep "
                        "ON gaia.source_id=cep.source_id",
                      dump_to_file=True, output_format='votable',)

In [ ]:
r = job.get_results()

ra=np.asarray(r['ra'].value)
dec=np.asarray(r['dec'].value)
par=np.asarray(r['parallax'].value)
par_err=np.asarray(r['parallax_error'].value)
mag=np.asarray(r['phot_g_mean_mag'].value)
ex=np.asarray(r['ag_gspphot'].value)
pf=np.asarray(r['pf'].value)
pf_err=np.asarray(r['pf_error'].value)
mode=np.asarray(r['mode_best_classification'].value)

sn=par/par_err

print('dr3 has ',par.shape,' cepheids')

b=np.asarray(r['b'].value)
l=np.asarray(r['l'].value)


In [ ]:
mode

Plot the position of cephieds

showing the positions of LMC and SMC, M31, M33

In [ ]:
lmc=SkyCoord.from_name('LMC')
smc=SkyCoord.from_name('SMC')
tri=SkyCoord.from_name('M33')
m31=SkyCoord.from_name('M31')

gal = SkyCoord(l, b, frame='galactic', unit=u.deg)
fig = plt.figure(figsize=(8,6))
ax = fig.add_subplot(111, projection='aitoff')
ax.grid(True)
ax.scatter(gal.l.wrap_at('180d').radian, gal.b.radian,s=0.5)
ax.scatter(lmc.galactic.l.wrap_at('180d').radian, lmc.galactic.b.radian,s=100,facecolors='none', edgecolors='r')
ax.scatter(smc.galactic.l.wrap_at('180d').radian, smc.galactic.b.radian,s=100,facecolors='none', edgecolors='r')
ax.scatter(tri.galactic.l.wrap_at('180d').radian, tri.galactic.b.radian,s=100,facecolors='none', edgecolors='r')
ax.scatter(m31.galactic.l.wrap_at('180d').radian, m31.galactic.b.radian,s=100,facecolors='none', edgecolors='r')

1. compute the seperation between our stars and LMC
2. mask out any stars more than 10deg from the LMC
3. replot

In [ ]:
cep = SkyCoord(ra, dec, frame='icrs', unit=u.deg)

sep = cep.separation(lmc)
msk=np.where((sep.deg<10.0) & (mode=='FUNDAMENTAL')) # & (par>0.0) & (ex>0.00001) & (ex<0.2) )

dist=Distance(parallax=par[msk]*u.mas,allow_negative=True)

fig = plt.figure(figsize=(8,6))
ax = fig.add_subplot(111, projection='aitoff')
ax.grid(True)
ax.scatter(gal[msk].l.wrap_at('180d').radian, gal[msk].b.radian,s=0.5)
ax.scatter(lmc.galactic.l.wrap_at('180d').radian, lmc.galactic.b.radian,s=100,facecolors='none', edgecolors='r')
#ax.scatter(smc.galactic.l.wrap_at('180d').radian, smc.galactic.b.radian,s=100,facecolors='none', edgecolors='r')

In [ ]:
period = pf[msk]
magnitude = mag[msk]
plt.scatter(period,magnitude,s=0.3,alpha=0.5)
plt.xscale('log')
plt.xlabel('period [days]')
plt.ylabel(r'$M_G$')

In [ ]:

job = Gaia.launch_job("SELECT gaia.source_id, gaia.ra, gaia.dec, gaia.b, gaia.parallax, gaia.parallax_error, cepheid.source_id, cepheid.pf, cepheid.pf_error, gaia.phot_g_mean_mag, gaia.phot_bp_mean_mag, gaia.phot_rp_mean_mag,gaia.phot_g_mean_flux_over_error, gaia.ag_gspphot , cepheid.mode_best_classification, gaia.distance_gspphot  "
                      "FROM gaiadr3.vari_cepheid AS cepheid "
                      "JOIN gaiadr3.gaia_source AS gaia ON "
                      " cepheid.source_id = gaia.source_id "
                      "WHERE cepheid.pf > 0.01 AND "
                      " gaia.ag_gspphot > 0.00001 AND "
                      " (cepheid.pf/cepheid.pf_error) > 10.0 AND "
                      " gaia.phot_g_mean_flux_over_error > 10.0   ",
                      dump_to_file=True, output_format='votable',)

1. Redo for the other magellanic clouds
2. Are the period-luminosity relations the same?

Lets look at the time series for Cepheids...

In [ ]:
query = f"SELECT TOP 10 \
gaia.source_id, gaia.ra, gaia.dec, gaia.parallax, \
variable.classification, variable.phot_variable_fundam_freq1, \
phot_stats.mean, \
cepheid.peak_to_peak_g, cepheid.num_harmonics_for_p1, cepheid.r21_g, cepheid.phi21_g, \
cepheid.type_best_classification, cepheid.type2_best_sub_classification, cepheid.mode_best_classification \
FROM gaiadr1.gaia_source AS gaia \
INNER JOIN gaiadr1.variable_summary AS variable \
ON gaia.source_id = variable.source_id \
INNER JOIN gaiadr1.phot_variable_time_series_gfov_statistical_parameters AS phot_stats \
ON gaia.source_id = phot_stats.source_id \
INNER JOIN gaiadr1.cepheid AS cepheid \
ON gaia.source_id = cepheid.source_id"
job = Gaia.launch_job_async(query,dump_to_file=True, output_format='votable',)

In [ ]:
results = job.get_results()
print(f'Table size (rows): {len(results)}')

In [ ]:
results

Lets choose a cepheid to investigate. First try ID=4658950381175117824 which is designated as FIRST_OVERTONE type of oscillation.

In [ ]:
query = f"SELECT \
curves.observation_time, \
mod(curves.observation_time - cepheid.epoch_g, cepheid.p1)/ cepheid.p1 as phase, \
curves.g_magnitude, \
2.5/log(10)* curves.g_flux_error/ curves.g_flux \
AS g_magnitude_error, \
rejected_by_variability_processing AS rejected \
FROM gaiadr1.phot_variable_time_series_gfov AS curves \
INNER JOIN gaiadr1.cepheid AS cepheid \
ON cepheid.source_id = curves.source_id \
WHERE cepheid.source_id = 4658950381175117824"
job = Gaia.launch_job_async(query,dump_to_file=True, output_format='votable',)

In [ ]:
results = job.get_results()
print(f'Table size (rows): {len(results)}')

In [ ]:
time = np.asarray(results["observation_time"])
mag = np.asarray(results["g_magnitude"])

In [ ]:
plt.scatter(time,mag)
plt.xlim((1665,1685))

Now try a FUNDAMENTAL mode cepheid and see how it looks...